In [6]:
from ase.io import read
from ase.cluster import Octahedron
from ase.optimize import LBFGS
from ase.visualize import view
from mace.calculators import MACECalculator
from acat.ga.adsorbate_operators import (AddAdsorbate, RemoveAdsorbate,
                                        MoveAdsorbate, ReplaceAdsorbate,
                                        SimpleCutSpliceCrossoverWithAdsorbates)
from acat.ga.particle_mutations import (RandomPermutation, COM2surfPermutation,
                                        Rich2poorPermutation, Poor2richPermutation)
from ase.ga.offspring_creator import OperationSelector
from acat.adsorption_sites import ClusterAdsorptionSites
from acat.adsorbate_coverage import ClusterAdsorbateCoverage

from npl.monte_carlo.ensembles.canonical_ensemble import CanonicalEnsemble

In [7]:
atoms = Octahedron('Ag', 6,1)
atoms.set_cell([25,25,25])
atoms.center()

sas = ClusterAdsorptionSites(atoms, ignore_sites=['ontop', 'bridge', 'longbridge', 'shortbridge'])
species = ['O']

add = AddAdsorbate(species, adsorption_sites=sas, num_muts=18, min_adsorbate_distance=2.7)
atoms.info['data'] = {'tag': None}
atoms.info['confid'] = 1
atoms, _ = add.get_new_individual([atoms])

view(atoms, viewer='x3d')

In [8]:
atoms=read('Ag140O30.xyz')
atoms.center(10)
atoms.set_pbc(False)
view(atoms, viewer='x3d')
sas = ClusterAdsorptionSites(atoms, ignore_sites=['ontop', 'bridge', 'longbridge', 'shortbridge'])
species = ['O']
calculator = MACECalculator(
    model_paths='/home/riccardo/Downloads/mace_agnesi_medium.model',
    device = 'cpu'
    )

view(atoms, viewer='x3d')

/home/riccardo/anaconda3/envs/np-lib/lib/python3.9/site-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
INFO - Using CPU


No dtype selected, switching to float64 to match model dtype.


In [5]:
soclist = ([1],[MoveAdsorbate(['O'], adsorption_sites=sas, num_muts=1)])
op_list = OperationSelector(*soclist)

outfile = f'{atoms.get_chemical_formula()}.out'
outfile_write_interval = 10
random_seed = 4234

montecarlo = CanonicalEnsemble(atoms=atoms,
                calculator=calculator,
                random_seed=random_seed,
                optimizer=LBFGS, 
                fmax=0.45,
                temperature=300,
                p=1,
                traj_file=f'{atoms.get_chemical_formula()}.traj', 
                op_list=op_list, 
                outfile=outfile,
                outfile_write_interval = outfile_write_interval)

montecarlo.run(steps=2000)

INFO - +---------------------------------+
INFO - | Canonical Ensemble Monte Carlo  |
INFO - +---------------------------------+
INFO - Starting simulation...
INFO - Temperature: 300
INFO - Number of steps: 2000
INFO - Rejected trial move with energy: -509.52848107152204
INFO - Rejected trial move with energy: -509.4792572426607
INFO - Rejected trial move with energy: -509.47989869907826
INFO - Rejected trial move with energy: -509.4618892310376
INFO - Rejected trial move with energy: -509.50402981776637
INFO - Rejected trial move with energy: -509.3907877347801
INFO - Rejected trial move with energy: -508.0357138272589
INFO - Rejected trial move with energy: -508.4688178351434
INFO - Accepted trial move with energy: -509.6218792329641
INFO - Rejected trial move with energy: -509.0654149239236
INFO - Step: 10
INFO - Lowest energy: -509.66332445202727
INFO - Accepted trials: 1
INFO - Rejected trial move with energy: -509.04945519340373
INFO - Rejected trial move with energy: -508.756704